In [1]:
import pandas as pd
df = pd.read_csv('./data/train-bio.csv', sep='\t', header=None)
df.columns = ['tokens','BIO_Label']

# df.isnull().sum() # 85 empty strings?
# df.dropna(inplace=True) #to remove these 85 values

essay = " "
for word in df['tokens']:
    essay = str(essay) + " " + str(word) + " "
    
print(len(essay))

806303


In [10]:
import pandas as pd
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn import metrics
from sklearn.metrics import confusion_matrix
#nltk.download('stopwords')

X,y = df['tokens'],df['BIO_Label']

corpus = []
for i in range(len(X)):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'\s+[a-z]\s+', ' ',review) #substitute all single characters with space on l&r
    review = re.sub(r'\s+', ' ', review)  #substitute all spaces to single space
    corpus.append(review) 

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(text_train,sent_train)
dtree_predictions = dtree_model.predict(text_test)
accuracy = dtree_model.score(text_test, sent_test)
print(accuracy)
print(metrics.classification_report(sent_test, dtree_predictions))
cm = confusion_matrix(sent_test, dtree_predictions)

df2 = pd.DataFrame(cm,index=['B','I','O'], columns=['B','I','O'])
print(df2)

0.6802661310569922


C:\Users\SAHILAGICHANI\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SAHILAGICHANI\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SAHILAGICHANI\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

              precision    recall  f1-score   support

           B       0.00      0.00      0.00       962
           I       0.67      1.00      0.81     15829
           O       1.00      0.06      0.11      7107

    accuracy                           0.68     23898
   macro avg       0.56      0.35      0.31     23898
weighted avg       0.74      0.68      0.57     23898

   B      I    O
B  0    962    0
I  0  15829    0
O  0   6679  428
